In [932]:
import pandas as pd
import numpy as np
import re

In [933]:
pd.set_option("display.max_columns", None)

In [934]:
df = pd.read_csv('mysmartprice_mobile_dataset.csv')


In [959]:
df.sample(5)

,brand,model,release_date,price,avg_rating,total_ratings,rear_camera,front_camera,display_size(inch),refresh_rate(Hz),display_type,battery_size(mAh),charging_speed(W),os_type,os_version,5g,nfc,fingerprint,expert_view,ram(gb),storage(gb),cpu_brand,cpu_model
1300,Asus,Rog phone 8 pro 1tb,2024-01-29,119999.0,NaN,NaN,50+13+32,32,6.78,165,Flexible AMOLED,5500.0,30.0,Android,v14,yes,yes,yes,NaN,24.0,0.000977,Snapdragon,8 Gen 3 Octa core
4058,Forme,Mini 11,2021-07-20,1149.0,4.0,1.0,0.3,NaN,2.8,NaN,TFT,1800.0,NaN,NaN,NaN,no,no,no,NaN,NaN,NaN,NaN,nan
2719,I Kall,Kall k200,2020-09-30,3599.0,3.4,17900.0,5,2,5.5,NaN,IPS LCD,2500.0,NaN,Android,v6.0,no,no,no,NaN,2.0,16.000000,NaN,Quad core
4983,I Kall,Kall z14,2022-08-09,6789.0,3.3,413.0,20,13,6.82,NaN,IPS LCD,5000.0,NaN,Android,v10,no,no,yes,The I Kall Z14 is a regular option in this pri...,4.0,64.000000,NaN,Quad core
2038,Panasonic,P100,2018-02-07,5299.0,3.8,2321.0,8,5,5.0,NaN,IPS LCD,2200.0,NaN,Android,v7.0,no,no,yes,NaN,1.0,16.000000,MediaTek,Quad core


#####
- drop col:['mobile_name','battery_and_charging_speed','operating_system','5G|NFC|Fingerprint','ram_and_storage']
- col:price -> price(INR)
- col:front_camera -> front_camera(MP)


In [936]:
df.drop(columns=['Unnamed: 0'],inplace=True)

In [961]:
df.info(show_counts=True)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5263 entries, 0 to 5262
Data columns (total 23 columns):
 #   Column              Non-Null Count  Dtype         
---  ------              --------------  -----         
 0   brand               5263 non-null   object        
 1   model               5262 non-null   object        
 2   release_date        5215 non-null   datetime64[ns]
 3   price               5251 non-null   float64       
 4   avg_rating          4700 non-null   float64       
 5   total_ratings       4700 non-null   float64       
 6   rear_camera         5110 non-null   object        
 7   front_camera        3494 non-null   object        
 8   display_size(inch)  5260 non-null   object        
 9   refresh_rate(Hz)    1526 non-null   object        
 10  display_type        4809 non-null   object        
 11  battery_size(mAh)   5236 non-null   float64       
 12  charging_speed(W)   1871 non-null   float64       
 13  os_type             3501 non-null   object      

In [938]:
#extract brand name seperately
df.insert(0,'brand',df['mobile_name'].str.split(' ', n=1).str[0].str.capitalize())   

In [939]:
#handle brand "I Kall"
df['brand'].replace('I', 'I Kall',inplace=True)

/var/folders/s_/8psmrzpd1ljgqhthwdx4fv000000gn/T/ipykernel_28851/4139014912.py:2: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  df['brand'].replace('I', 'I Kall',inplace=True)


In [940]:
#extract model seperately
df.insert(1,'model',df['mobile_name'].str.split(' ', n=1).str[1].str.capitalize()) 

In [941]:
#convert release_date to datetime
df['release_date'] = pd.to_datetime(df['release_date'].str.split(':').str[1])

In [942]:
#convert ratings column to float
def convert_ratings(value):
    if pd.isna(value):  # Handle NaN values
        return np.nan
    
    value = value.split(' ')[0]

    if 'K' in value:
        return (float(value.replace(',','').replace('K',''))*1000)
    elif 'L' in value:
        return (float(value.replace(',','').replace('L',''))*100000)
    else:
        return (float(value.replace(',','')))

df['total_ratings'] = df['total_ratings'].astype(str).apply(convert_ratings)

In [943]:
#change price to float
df['price'] = df['price'].str.replace('₹','').str.replace(',','').astype(float)

In [944]:
#clean col : front camera
df['front_camera'] = df['front_camera'].str.replace('MP Front Camera','')

In [945]:
#clean col : rear camera
df['rear_camera'] = df['rear_camera'].str.replace('MP Rear Camera','')

In [946]:
#create new col: battery_size(mAh)
df.insert(12,'battery_size(mAh)',df['battery_and_charging_speed'].str.split(' ',n=1).str[0].astype(float))

In [947]:
#create new col : charging_speed(W)
df.insert(13,'charging_speed(W)',df['battery_and_charging_speed'].str.split('|',n=1).str[1].str.strip().str.split('W').str[0].astype(float))

In [948]:
#operating system -> create new cols:os_type,os_version
df['operating_system'] = df['operating_system'].str.replace(' OS','',)
df.insert(15,'os_type',df['operating_system'].str.split(' ',n=1).str[0])
df.insert(16,'os_version',df['operating_system'].str.split(' ',n=1).str[1])

In [949]:
#add cols : 5g,nfc,fingerprint
df.insert(18,'5g',df['5G|NFC|Fingerprint'].astype(str).apply(lambda x: 'yes' if '5G' in x else 'no'))
df.insert(19, 'nfc', df['5G|NFC|Fingerprint'].astype(str).apply(lambda x: 'yes' if 'NFC' in x else 'no'))
df.insert(20, 'fingerprint', df['5G|NFC|Fingerprint'].astype(str).apply(lambda x: 'yes' if 'Fingerprint' in x else 'no'))


In [952]:
import numpy as np
import pandas as pd

def extract_size(value):
    """Extracts numeric size and converts MB to GB if needed."""
    parts = value.strip().split(' ')  # Extract number and unit
    num = float(parts[0])  # Convert to float
    return num if 'GB' in value else num / 1024  # Convert MB to GB

def ram_and_storage(value):
    """Extracts RAM and Storage values separately in GB."""
    if pd.isna(value):  # Handle NaN values
        return np.nan, np.nan

    ram, storage = np.nan, np.nan  # Default values
    parts = value.split('|')  # Split RAM and Storage

    for part in parts:
        part = part.strip()
        if 'RAM' in part:
            ram = extract_size(part)
        elif 'Storage' in part:
            storage = extract_size(part)

    return ram, storage

# Apply function and create new columns
df[['ram(gb)', 'storage(gb)']] = df['ram_and_storage'].astype(str).apply(lambda x: pd.Series(ram_and_storage(x)))


In [953]:
#add col:display_size(inch)
df.insert(10,'display_size(inch)',df['display'].str.split('″').str[0])

In [954]:
#add col :refresh_rate(Hz)
df.insert(11,'refresh_rate(Hz)',df['display'].astype(str).apply(lambda x : (re.search(r'(\d+)(\s*)Hz',x).group(1)) if re.search(r'(\d+)(\s*)Hz',x) else np.nan))


In [955]:
#add col:display_type
display_types = ['TFT', 'TFT LCD', 'TFT LTPS' ,'TFD', 
 'IPS LCD', 'IPS LED', 'IPS Plus LCD',
 'HD IGZO LCD', 'S-LCD 5', 'LCD','PLS LCD','S-LCD' ,'S-LCD 3','S-LCD 2', 'AHVA LCD', 'LTPS LCD'
 'AMOLED' , 'Super AMOLED', 'Optic AMOLED', 'Dynamic AMOLED', 'Fluid AMOLED', 'Flexible AMOLED', 'Flexi-fluid AMOLED', 'Super AMOLED Plus', 'LTPO AMOLED', 'LTPS AMOLED', 'Super Fluid AMOLED',
 'P-OLED','OLED', 'LTPO OLED', 
 'Super Retina XDR','ProXDR LTPO']

def extract_display_type(text):

    for display in display_types:
        if re.search(rf'\b{re.escape(display)}\b', text, re.IGNORECASE):
            return display
    return np.nan  # Return None if no match found

df.insert(12,'display_type',pd.Series(df['display'].astype(str).apply(extract_display_type)))

In [957]:
brands = ['Snapdragon', 'Samsung', 'MediaTek', 'Apple', 'Unisoc', 'Google', 'HiSilicon', 'Spreadtrum', 'ST-Ericsson', 'Marvell', 'Huawei', 'Broadcom', 'Intel', 'Nvidia']

def cpu_brand(value):
    
    #If nan value
    if value == np.nan:
        return np.nan , np.nan
    
    for brand in brands:
        if brand in value:
            print(value)
            if len(value.split())>1:
                return value.split(' ')[0] , ' '.join(value.split(' ')[1:])
            else:
                return np.nan,value
    
    return np.nan,value


df[['cpu_brand','cpu_model']] = df['cpu'].astype(str).apply(lambda x: pd.Series(cpu_brand(x)))

Snapdragon 8 Elite Octa core
Snapdragon 8 Elite Octa core
Snapdragon 8 Elite Octa core
Samsung Exynos 1380 Octa core
Snapdragon 7s Gen 2 Octa core
Snapdragon 6s Gen 3 Octa core
Snapdragon 7 Gen 3 Octa core
Snapdragon 8 Gen 3 Octa core
Snapdragon 7 Gen 3 Octa core
Snapdragon 7s Gen 3 Octa core
Snapdragon 8 Gen 3 Octa core
Samsung Exynos 1380 Octa core
Snapdragon 8 Elite Octa core
Snapdragon 7 Gen 3 Octa core
Snapdragon 7 Plus Gen 3 Octa core
MediaTek Dimensity 8400 Ultra Octa core
Snapdragon 7 Gen 3 Octa core
MediaTek Dimensity 9400 Octa core
MediaTek Dimensity 7025 Ultra Octa core
MediaTek Dimensity 7300 Octa core
MediaTek Dimensity 8350 Octa core
MediaTek Dimensity 7300 Octa core
MediaTek Dimensity 7300 Ultra Octa core
Snapdragon 7s Gen 2 Octa core
MediaTek Dimensity 6300 Octa core
MediaTek Dimensity 7300 Octa core
Snapdragon 7 Plus Gen 3 Octa core
Snapdragon 7 Gen 3 Octa core
Snapdragon 8 Elite Octa core
Snapdragon 6 Gen 1 Octa core
MediaTek Dimensity 7300 Ultra Octa core
Samsung Exy

In [958]:
df.drop(columns = ['mobile_name','battery_and_charging_speed','operating_system','5G|NFC|Fingerprint','ram_and_storage','display','display','cpu'],inplace = True)

In [962]:
final_df=df[['brand', 'model', 'release_date', 'price', 'avg_rating','total_ratings', 'rear_camera', 'front_camera', 'display_size(inch)','display_type','refresh_rate(Hz)','ram(gb)', 'storage(gb)','battery_size(mAh)', 'cpu_brand', 'cpu_model','charging_speed(W)', 'os_type', 'os_version', '5g', 'nfc','fingerprint', 'expert_view']]

In [964]:
final_df.to_csv('mysmartprice_mobile_dataset_cleaned.csv')